# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import sys
import math
import numpy as np
from typing import List
from collections import namedtuple

# Basic data structures.
Pose = namedtuple("Pose", ["x", "y", "theta"])  # x, y coordinate and the rotation of the robot.
Landmark = namedtuple("Landmark", ["x", "y"])   # x, y coordinate of the landmark.
Measurement = namedtuple("Measurement", ["r", "b"])  # r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Command = namedtuple("Command", ["delta_p", "delta_theta"])


def eval_gaussian(x, mean=0, var=1):
  """Compute the Gaussian pdf function."""
  return 1.0/math.sqrt(2*math.pi*var)*math.exp(-0.5*(x-mean)**2/var)


class Simulator:
  robot: Pose
  motion_noise_covariance: np.ndarray  # 2x2 covariance matrix of the motion noise.
  sensor_noise_covariance: np.ndarray  # 2x2 covariance matrix of the sensor noise.
  landmark: List[Landmark]  # a list of landmarks.

  def __init__(self):
    self.robot = Pose(0, 0, 0)
    self.motion_noise_covariance = np.array([[1, 0], [0, 1]], dtype='float64')
    self.sensor_noise_covariance = np.array([[1, 0], [0, np.deg2rad(5)**2]], dtype='float64')
    self.landmarks = [Landmark(-1, -1), Landmark(-1, -1), Landmark(11, 11), Landmark(-1, 11)]

  def set_motion_noise_covariance(self, cov):
    """Set the motion noise covariance."""
    self.motion_noise_covariance = np.array(cov, dtype='float64')

  def set_sensor_noise_covariance(self, cov):
    """Set the sensor noise covariance."""
    self.sensor_noise_covariance = np.array(cov, dtype='float64')

  def simulate_motion(self, command):
    """Simulate robot motion. This function updates the value of self.robot.

    Args:
      command: a Command tuple containing fields delta_p(float), the distance of the
      movement, and delta_theta(float), the rotation of the movement.
    """
    noise = np.random.multivariate_normal([0, 0], self.motion_noise_covariance)
    x = self.robot.x + math.cos(self.robot.theta) * (command.delta_p + noise[0])
    y = self.robot.y + math.sin(self.robot.theta) * (command.delta_p + noise[0])
    theta = self.robot.theta + command.delta_theta + noise[1]
    self.robot = Pose(x, y, theta)

  def simulate_sensing(self):
    """Simulate the robot sensing process. This function returns a list of measurements, in the same order as the
    self.landmarks list. Specifically, it measures the distance and the angle w.r.t. each landmark, and add a
    Gaussian noise on the measurement.

    Returns:
      measurements: a list of measurements, in the same order as the self.landmarks list.
    """

    measurements = []
    for landmark in self.landmarks:
      noise = np.random.multivariate_normal([0, 0], self.sensor_noise_covariance)

      r = math.hypot(landmark[0]-self.robot.x, landmark[1]-self.robot.y) + noise[0]
      b = math.atan2(landmark[1]-self.robot.y, landmark[0]-self.robot.x) + noise[1]

      measurements.append(Measurement(r, b))

    return measurements


def drive_in_a_square(simulator):
  """Simulate the robot's movement in a square. See the code for details.

  Returns:
    poses: a numpy array of the robot's trajectory. It is a T by 3 matrix, where T is the trajectory length, and the
    second dimension contains: x, y, theta.
  """
  poses = []
  poses.append(simulator.robot)
  for sides in range(4):
    for i in range(100):
      simulator.simulate_motion(Command(.1, 0))
      poses.append(simulator.robot)
    simulator.simulate_motion(Command(0, np.pi/2))
    poses.append(simulator.robot)
  return np.array(poses)


def press(event):
  """matplotlib helper function. It processes the keyboard event. If the user hits q, it exit the program."""
  import matplotlib.pyplot as plt
  if event.key == 'q':
    sys.exit(0)
  plt.close()


def plot_samples(samples):
  """matplotlib helper function. It takes a list of samples as input and scatter plot the samples.

  Args:
    samples: a numpy array of the samples. This function only plots the first two components: x and y.
  """
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots()
  ax.axis([-2, 12, -2, 12])
  fig.canvas.mpl_connect('key_press_event', press)

  ax.set_xlabel('x')
  ax.set_ylabel('y')
  ax.plot(samples[:, 0], samples[:, 1], 'g+')
  plt.show()


def plot_trajectories(true_poses, estimated_poses):
  """matplotlib helper function. It overlays both the true pose and the estimated pose.

  Args:
    true_poses: a numpy array of the true robot trajectory. This function only plots the first two components: x and y.
    estimated_poses: a numpy array of the estimated robot trajectory. This function only plots the first two components: x and y.
  """

  import matplotlib.pyplot as plt
  fig, ax = plt.subplots()
  ax.axis([-20, 20, -20, 20])
  fig.canvas.mpl_connect('key_press_event', press)

  ax.set_xlabel('x')
  ax.set_ylabel('y')
  ax.plot(true_poses[:, 0], true_poses[:,1], 'b')
  ax.plot(estimated_poses[:, 0], estimated_poses[:,1], 'r')
  plt.show()




## Problems

### Warmup 1
Let's make sure we know how to resample a set of
weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.choice.


For reference, our solution is **1** lines of code.

In [ ]:
def warmup_1(samples, weights, nr_samples):
  """Draw N samples from the input list based on their weights.

  Args:
    samples: an input list.
    weights: a numpy list of the same length as samples, indicating the weights for individual samples.
    nr_samples: an integer.

  Returns:
    resampled_samples: nr_samples samples drawn from the input list, based on the weights with replacement.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.03191091, 0.10365558, 0.07293406, 0.13085621, 0.12995933, 0.0454193 , 0.04606439, 0.1336077 , 0.15964489, 0.14594763], dtype=np.float64), 10), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4], dtype=np.int64), atol=1e-6)
print('Tests passed.')

### Warmup 2
We are going to be sampling from Gaussian
systems in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.multivariate_normal.


For reference, our solution is **1** lines of code.

In [ ]:
def warmup_2(mean, covariance, num_samples):
  """Sample from a multi-variate Gaussian distribution.

  Args:
    mean: a vector in numpy.
    covariance: a matrix in numpy.
    num_samples: an integer.

  Returns:
    sample: a numpy vector with length num_samples.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([0.58345008, 0.05778984, 0.13608912], dtype=np.float64), np.array([[1.61405906, 0.93399482, 0.79165733],  [0.93399482, 1.65599579, 1.43616931],  [0.79165733, 1.43616931, 1.25177385]], dtype=np.float64), 5), np.array([[-0.8347847 , -2.30256418, -1.83805061],  [-0.13115218, -3.33917428, -2.91354857],  [-0.47476854, -1.05885092, -0.83435982],  [ 0.29510331, -0.55660856, -0.28675913],  [-0.06919861, -0.94320128, -0.69794801]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 1
Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

**Hint**: you can use `numpy.random.multivariate_normal` to sample the noise.


For reference, our solution is **5** lines of code.

In [ ]:
def motion_model(samples, command, motion_noise_covariance):
  """A motion model that simulates a one-step movement of the robot,
  and updates the samples.

  Args:
    samples: a list of samples, given as a numpy array.
    command: a Command tuple containing fields delta_p(float), the distance of the
    movement, and delta_theta(float), the rotation of the movement.
    motion_noise_covariance: a 3x3 numpy array for the motion covariance.

  Returns:
    new_samples: a numpy array of new samples by applying the motion model.
  """
  new_samples = list()
  for sample in samples:
    # generate the noise.
    noise = np.random.multivariate_normal([0, 0], motion_noise_covariance)
    # TODO (YOUR CODE):: compute the updated sample pose and append it to new_smaples.

  return np.array(new_samples)

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(motion_model(np.array([[ 0.14940791,  0.03336743, -0.01790344],  [-0.08540957,  0.03130677, -0.22279039],  [ 0.08644362,  0.06536186, -0.06476612],  [-0.14543657,  0.22697546,  0.00399318],  [ 0.15327792, -0.01871839,  0.12822574],  [ 0.03781625,  0.01549474, -0.07747392],  [-0.03479121, -0.19807965,  0.01364402],  [ 0.12023798,  0.12302907, -0.03380064],  [-0.1048553 , -0.03023028, -0.12391994],  [ 0.19507754, -0.17062702, -0.04447554]], dtype=np.float64), Command(1, 0), [[0.01, 0], [0, 0.007615435494667714]]), np.array([[ 1.32562461,  0.01230686,  0.01701687],  [ 0.98532995, -0.21127054, -0.02723557],  [ 1.27071128, -0.01144598, -0.1500497 ],  [ 0.94956354,  0.23134802, -0.00921522],  [ 1.13483111,  0.10783636,  0.16405722],  [ 1.0491778 , -0.06301655,  0.04943538],  [ 1.0412124 , -0.18339772,  0.02426217],  [ 1.16402777,  0.08773486, -0.00468206],  [ 1.03573866, -0.17230058, -0.14182338],  [ 1.22536448, -0.21647983, -0.11900946]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 2
Let us now add in a measurement model. Let us assume that
there are four landmarks located at locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
bearing [r, b] from the robot's location to each of these landmarks. Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Given a set of measurements, we can use this model to compute the importance
weights of a set of samples of the robot pose.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.


For reference, our solution is **16** lines of code.

In [ ]:
def compute_weights(samples, measurement, landmarks, sensor_noise_covariance):
  """Compute the importance weights of the samples, based on the new measurement.

  Args:
    samples: a list of samples.
    measurement: a measurement vector. The measurement is computed by
      simulator.simulate_sensing() function.
    landmarks: a list of Landmark instances. They are namedtuples of (x, y).
    sensor_noise_covariance: a 2x2 numpy array for the sensor noise for the
    measurements of range and bearing to the landmarks. See the doc for
    class `Measurement` and `simulator.simulator_sensing`

  Returns:
    weights: a numpy array of importance weights.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(compute_weights(np.array([[ 0.06536186, -0.25529898,  0.07543629],  [ 0.22697546, -0.0742165 , -0.12691735],  [-0.01871839,  0.00457585,  0.13376022],  [ 0.01549474,  0.14693588,  0.03300091],  [-0.19807965, -0.08877857, -0.03036106],  [ 0.12302907,  0.0156349 ,  0.10492744],  [-0.03023028, -0.03873268, -0.09150351],  [-0.17062702, -0.14200179,  0.17023727],  [-0.04380743, -0.05096522, -0.10932702],  [-0.16138978,  0.07774904, -0.01856509]], dtype=np.float64), [Measurement(r=0.5187470011794195, b=-2.3224308778331726), Measurement(r=0.9034084248042221, b=-2.4592240845320847), Measurement(r=15.52816695776539, b=0.8227771705573576), Measurement(r=11.111878239570428, b=1.6878518665472155)], [Landmark(x=-1, y=-1), Landmark(x=-1, y=-1), Landmark(x=11, y=11), Landmark(x=-1, y=11)], np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64)), np.array([0.01711671, 0.04172281, 0.14431346, 0.04618495, 0.05960219, 0.16749519, 0.17919444, 0.15658841, 0.18128191, 0.00649993], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Main Question
Now let's put everything together. We've given you a method
test_localization() that does the following:
  * Initializes a set of samples drawn from a Gaussian distribution with mean
equal to the robot starting position, and a diagonal covariance matrix equal
to np.diag(1, 1, np.deg2rad(5))^2.
  * Drives the robot along the same pattern as in the drive_in_a_square method,
but on each step, after the command motion do the following:
    * Simulates measurements.
    * Update the samples based on the measurements.

What we want you to do is to write a method for step 4: `update_samples` that takes in a
set of samples, the most recent command, the most recent measurement, and the
motion and sensor covariances and does the following:
  * use the motion_model method to update the sample based on the command
  * use compute_weights to compute the sample weights from the
measurement,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag(.001, .001, np.deg2rad(1))^2`.


For reference, our solution is **14** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
def update_samples(samples, command, measurement, landmarks, motion_noise_covariance, sensor_noise_covariance):
  """Update the samples, based on the command and the new measurement.

  Args:
    samples: a list of samples.
    command: a Command tuple containing fields delta_p(float), the distance of the
    movement, and delta_theta(float), the rotation of the movement.
    measurement: a measurement vector. The measurement is computed by
      simulator.simulate_sensing() function.
    landmarks: a list of Landmark instances. They are namedtuples of (x, y).
    motion_noise_covariance: a 3x3 numpy array for the motion covariance.
    sensor_noise_covariance: a 2x2 numpy array for the sensor noise for the
    measurements of range and bearing to the landmarks

  Returns:
    weights: a numpy array of importance weights.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
ret = update_samples(np.array([[ 0.12654082,  0.55784233,  0.085411  ],  [ 0.59057369,  0.70863265, -0.08528358],  [-0.04786335,  0.30044434, -0.00900754],  [ 0.04555058,  0.12984265,  0.1269093 ],  [ 0.03847702,  0.24066126,  0.03873437],  [ 0.47246929,  0.10551709, -0.01790344],  [-0.27008879,  0.0990007 , -0.22279039],  [ 0.27335873,  0.20669235, -0.06476612],  [-0.45991081,  0.71775943,  0.00399318],  [ 0.48470735, -0.05919273,  0.12822574]], dtype=np.float64), Command(1, 0.7853981633974483), [Measurement(r=1.4878822442234976, b=-2.879974750986568), Measurement(r=2.0277558425274576, b=-2.693473628609837), Measurement(r=15.992575740663117, b=0.9456753162622136), Measurement(r=10.821767633417135, b=1.7378697490164574)], [Landmark(x=-1, y=-1), Landmark(x=-1, y=-1), Landmark(x=11, y=11), Landmark(x=-1, y=11)], np.array([[1., 0.],  [0., 1.]], dtype=np.float64), np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64))
target = np.array([[ 1.66279472,  0.13350661,  2.22447258],  [ 1.75892096,  0.12067389,  0.02495675],  [ 1.45236856, -0.30110589,  0.70575541],  [ 1.43517602, -0.25208135,  0.67900661],  [ 1.40251946, -0.3232621 ,  0.65450276],  [ 1.69807815,  0.17493323,  1.04666054],  [ 1.47201077, -0.32972081,  0.65611496],  [ 1.41910723, -0.29683138,  0.69103551],  [ 1.57899464,  0.06888279,  2.22127636],  [ 1.44952783, -0.2765589 ,  0.68956192]], dtype=np.float64)
assert all((np.allclose(x, y, atol=1e-6) for x, y in zip(ret, target)))
print('Tests passed.')

Below is our implementation of test_localzation(). Please use this function to run the tests and submit your findings.

In [ ]:
def test_localization(simulator):
  num_samples = 10  # number of initial samples.
  mean = simulator.robot
  covariance = [[.1, 0, 0], [0, .1, 0], [0, 0, np.deg2rad(5)**2]]
  samples = np.random.multivariate_normal(mean, covariance, num_samples)

  true_poses = []
  true_poses.append(simulator.robot)
  estimated_poses = []
  estimated_poses.append(mean)

  for sides in range(4):
    # Straight line motion
    for i in range(10):  # we have lowered the resolution to make the code run faster.
      command = Command(1, 0)

      # Step 1: Simulate the robot motion and append it to true_poses.
      simulator.simulate_motion(command)
      true_poses.append(simulator.robot)
      measurement = simulator.simulate_sensing()

      # Step 2: Compute the estimated motion based on the commanded motion and
      # received measurement using update_samples and then append the mean of
      # the samples to estimated_poses.
      samples = update_samples(samples, command, measurement,
                               simulator.landmarks,
                               simulator.motion_noise_covariance,
                               simulator.sensor_noise_covariance)
      estimated_poses.append(np.mean(samples, axis=0))

    # Turn left 90 degrees
    command = Command(0, np.deg2rad(90))

    # Step 1: Simulate the robot motion and append it to true_poses.
    simulator.simulate_motion(command)
    true_poses.append(simulator.robot)
    measurement = simulator.simulate_sensing()

    # Step 2: Compute the estimated motion based on the measurement and append it to estimated_poses.
    # You can use the resample helper function to resample and add noise.
    samples = update_samples(samples, command, measurement,
                             simulator.landmarks,
                             simulator.motion_noise_covariance,
                             simulator.sensor_noise_covariance)
    estimated_poses.append(np.mean(samples, axis=0))

  return np.array(true_poses), np.array(estimated_poses)